## Ingesting green_taxi_trips

In [1]:
import pandas as pd
import requests
from sqlalchemy import create_engine
import gzip
import shutil

In [2]:
url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz"

user = "postgres"
password = "postgres"
db = "ny_taxi"
host = "localhost"
port = 5433

table_name = "green_taxi_trips_201910"

In [3]:
response = requests.get(url)
if response.status_code == 200:
    with open("green_tripdata_2019-10.csv.gz", "wb") as f_out:
        f_out.write(response.content)
else:
    print(f"Error code: {response.status_code}")

with gzip.open("green_tripdata_2019-10.csv.gz", "rb") as f_in:
    with open("green_trip_201910.csv", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [4]:
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db}")

In [5]:
df_iter = pd.read_csv("green_trip_201910.csv", iterator=True, chunksize=100000)
df = next(df_iter)

df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

In [6]:
df.head(n=0).to_sql(name=table_name, con=engine, if_exists="replace")

## ingesting the fhrst chunk of data
df.to_sql(name=table_name, con=engine, if_exists="append")

1000

In [7]:
## ingesting the remaining data 

while True:
    try:
        df = next(df_iter)
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
        df.to_sql(name=table_name, con=engine, if_exists="append")
        print("Inserted another chunk")
    except:
        print("Insertion Completed")
        break

Inserted another chunk
Inserted another chunk


/tmp/ipykernel_34435/610660675.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = next(df_iter)


Inserted another chunk
Inserted another chunk
Insertion Completed


### Ingesting zones data

In [8]:
url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv"

In [9]:
user = "postgres"
password = "postgres"
db = "ny_taxi"
host = "localhost"
port = 5433

In [10]:
table_name = "zones"

In [11]:
response = requests.get(url)
if response.status_code == 200:
    with open("zones.csv", "wb") as f_out:
        f_out.write(response.content)
else:
    print(f"Error code: {response.status_code}")

In [12]:
df = pd.read_csv("zones.csv")

In [13]:
df.dtypes

LocationID       int64
Borough         object
Zone            object
service_zone    object
dtype: object

In [14]:
df.head(n=0).to_sql(name=table_name, con=engine, if_exists="replace")

0

In [15]:
df.to_sql(name=table_name, con=engine, if_exists="append")

265